In [1]:
!pip install face_recognition
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import cv2
import numpy as np
import face_recognition
import os

In [3]:
# importing images
path = 'Images'
# list of images
images = []
# writing names of each image
classNames = []
# pathing list of images in folder
myList = os.listdir(path)

# importing images one by one
for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')  # reading images from path folder cl is name of image
    images.append(curImg)
    # appending class name and converting abhinav .jpg to abhinav using splitext
    classNames.append(os.path.splitext(cl)[0])

In [4]:
# computing encoding for each image
def findEncodings(images):
    # creating empty list for encoding
    encodeList = []
    # looping all images
    for img in images:
        # converting image to rgb
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # finding encodings
        encode = face_recognition.face_encodings(img)[0]
        # appending it to the list
        encodeList.append(encode)
    return encodeList

In [5]:
# calling findEncodings method
encodeListKnown = findEncodings(images)
# print(len(encodeListKnown))
print('Encoding Complete')

Encoding Complete


In [6]:
# initialize webcam to capture image
cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read() # it will give image
    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25) # compressing image
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB) # converting it to rgb
    # finding location of faces
    facesCurFrame = face_recognition.face_locations(imgS)
    # finding encoding of images
    encodeCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)

    # iterating to all faces and comparing with webcam image
    # it will grab one face location from facesCurFrame list And it will grab encoding encodeFace from encodeCurFrame
    for encodeFace, faceLoc in zip(encodeCurFrame, facesCurFrame):
        # matching/comparing images with encodeFace
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
        # finding distance
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)

        # print(faceDis) # will return list with distances And the lowest distance is the best matching
        # finding the lowest element index in list
        matchIndex = np.argmin(faceDis)

        # display rectangle box around face and name them
        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            # print(name)
            # finding locations of face
            y1, x2, y2, x1 = faceLoc
            # decompressing image
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            # creating rectangle box
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2) # (0, 255, 0) color, thickness 2
            # creating other rectangle for name
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
            # putting name
            cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)

    # showing image
    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

        

KeyboardInterrupt: 